# Project: Profitable App Profiles for the App Store and Google Play Markets
Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

## Opening and Exploring the Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.
![data_numbers](https://s3.amazonaws.com/dq-content/350/py1m8_statista.png)
Sources:[Statista]((https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, we should first try to see if we can find any relevant existing data at no cost. Luckily, here are two data sets that seem suitable for our goals:

- [A database](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)
- [A dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv)

Let's start by **opening and exploring** these data sets.

Define the `open_file()` function with built-in functions

In [1]:
def open_file(file_path):
    opened_file=open(file_path, encoding='utf8')
    from csv import reader
    read_file=reader(opened_file)
    data=list(read_file)
    return data

applestore=open_file('AppleStore.csv')
googleplay=open_file('googleplaystore.csv')

Then we'll define and call the `explore_data()` function to help us explore the data

In [2]:
def explore_data(dataset, start, end, rows_and_columns=True):
    data_slice=dataset[start:end]
    for row in data_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns', len(dataset[0]),'\n')

print(applestore[0],'\n')
explore_data(applestore,1,5)

print(googleplay[0],'\n')
explore_data(googleplay,1,5)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows:  7198
Number of columns 16 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo 

In the previous step, we opened the two data sets and explored the data. Before beginning our analysis, we need to make sure the data we analyze is accurate, or the results of our analysis will be wrong. This means that we need to do the following:
- Detect inaccurate data, and correct or remove it.
- Detect duplicate data, and remove the duplicates.

And let's get into the **data cleaning** process.

## Cleaning Data

**1. Removing Inaccurate Data**

Before beginning our analysis, we need to make sure the data we analyze is accurate, or the results of our analysis will be wrong. This means that we need to detect and delete wrong data.

Notice: Execute once or run the entire code again

In [3]:
def find_row_missing(dataset): #header considered
    count=0
    for row in dataset[1:]:
        if len(row)!=len(dataset[0]):
            print(row)
            print('Index number: ', str(dataset.index(row)), '\n')
            del dataset[dataset.index(row)]
            count+=1
    print('There are ', str(count), ' missing rows and it\'s already been deleted')

find_row_missing(googleplay)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
Index number:  10473 

There are  1  missing rows and it's already been deleted


**2. Romoving Duplicate App Entries**

If we explore the Google Play data set long enough, we'll notice some apps have duplicate entries. For instance, Instagram has four entries:

In [4]:
for app in googleplay:
    name=app[0]
    if name=='Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In total, there are **1,181** cases where an app occurs more than once which means the number of clean data should be **9659(without header)**:

In [5]:
duplicate_apps=[]
unique_apps=[]
for app in googleplay[1:]:
    if app[0] in unique_apps:
        duplicate_apps.append(app[0])
    else:
        unique_apps.append(app[0])

print('Number of duplicate apps: ', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps: ', duplicate_apps[:15])
print('\n')
print('Expected length: ', len(googleplay[1:])-1181)

Number of duplicate apps:  1181


Examples of duplicate apps:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Expected length:  9659


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.


If we examine the rows we printed for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times.


We can use this information to build a criterion for removing the duplicates. The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

To remove the duplicates, we will create the `del_dup()` function below:

In [6]:
def del_dup(dataset,criteria):    #header cinsidered
    reviews_max={}
    for row in dataset[1:]:
        name=row[0]
        n_reviews=float(row[criteria])
        if name not in reviews_max:
            reviews_max[name]=n_reviews
        if name in reviews_max and n_reviews>reviews_max[name]:
            reviews_max[name]=n_reviews
        
    clean_dataset=[]
    already_added=[]
    for row in dataset[1:]:
        name=row[0]
        n_reviews=float(row[criteria])
        if n_reviews==reviews_max[name] and name not in already_added:
            clean_dataset.append(row)
            already_added.append(name)
    
    return clean_dataset #without header

In [7]:
googleplay_clean=del_dup(googleplay,3)
print(len(googleplay_clean))

9659


In the previous step, we managed to remove the duplicate app entries in the `googleplay` dataset. We don't need to do the same for the `applestore` data because there are no duplicates — you can check that for yourself using the `id column` (not the `track_name` column).

However, in order to keep both dataset in the same structure, we'll also create a list called `appstore_clean` which only contains data without a header.

In [8]:
applestore_clean=applestore[1:]

**3. Removing Non-English Apps**

Remember we use English for the apps we develop at our company, and we'd like to analyze only the apps that are designed for an English-speaking audience. However, if we explore the data long enough, we'll find that both datasets have apps with names that suggest they are not designed for an English-speaking audience.

In [9]:
print(applestore_clean[813][1])
print(applestore_clean[6731][1])
print('\n')
print(googleplay_clean[4412][0])
print(googleplay_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

This solution should work fine, but some English app names use emojis or other symbols like `''Docs To Go™ Free Office Suite'` and `'Instachat 😜'` will fall outside of the ASCII range.

To minimize the impact of data loss, we'll only identify an app as non-English if its name has more than three non-ASCII characters:

In [10]:
def is_eng(string):
    out_eng=0
    for word in string:
        if ord(word) > 127:
            out_eng+=1
    if out_eng>3:
        return False
    else:
        return True
    
print(is_eng('Docs To Go™ Free Office Suite'))
print(is_eng('Instachat 😜'))
print(is_eng('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


Now let's use the `is_eng()` function to filter out the non-English apps for both data sets:

In [11]:
ios_eng=[]
android_eng=[]

for row in applestore_clean:
    name=row[1]
    if is_eng(name):
        ios_eng.append(row)
        
for row in googleplay_clean:
    name=row[0]
    if is_eng(name):
        android_eng.append(row)
        
explore_data(ios_eng,0,3)
print('\n')
explore_data(android_eng,0,3)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows:  6183
Number of columns 16 



['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Va

**4. Isolation the Free Apps**

In [12]:
def free_app(dataset,index):
    free_apps=[]
    for row in dataset:
        price=row[index]
        if price=='0' or price=='0.0':
            free_apps.append(row)
    return free_apps

ios_final=free_app(ios_eng, 4)
android_final=free_app(android_eng, 7)

print(len(ios_final))
print(len(android_final))

3222
8864


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

## Most Common Apps by Genre

As we mentioned in the introduction, our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:
- Build a minimal Android version of the app, and add it to Google Play.
- If the app has a good response from users, we develop it further.
- If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by determining the most common genres for each market.

**1. Identifying the Columns for genres**

In [13]:
print(ios_final[0]) #11
print('\n')
print(android_final[0]) #1

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


**2. Building the Frequency Table and the Displaying Table**

In [14]:
def freq_table(dataset, index):
    result={}
    for row in dataset:
        name=row[index]
        if name in result:
            result[name]+=1
        if name not in result:
            result[name]=1
    return result

def display_table(dataset, index):
    table=freq_table(dataset, index)
    table_display=[]
    for key in table:
        as_tuple=(table[key], key)
        table_display.append(as_tuple)
    
    sorted_table=sorted(table_display, reverse=True)
    for item in sorted_table:
        percent=round((item[0]/len(dataset))*100,1)
        print(item[1], ':', percent, '%')

**3. Analyzing the Data**

We'll start by analyzing the `prime_genre` column of the `ios_final` dataset.

In [15]:
display_table(ios_final, 11)

Games : 58.2 %
Entertainment : 7.9 %
Photo & Video : 5.0 %
Education : 3.7 %
Social Networking : 3.3 %
Shopping : 2.6 %
Utilities : 2.5 %
Sports : 2.1 %
Music : 2.0 %
Health & Fitness : 2.0 %
Productivity : 1.7 %
Lifestyle : 1.6 %
News : 1.3 %
Travel : 1.2 %
Finance : 1.1 %
Weather : 0.9 %
Food & Drink : 0.8 %
Reference : 0.6 %
Business : 0.5 %
Book : 0.4 %
Navigation : 0.2 %
Medical : 0.2 %
Catalogs : 0.1 %


We can see that among the free English apps, more than a half (55.6%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 4%. Only 3.5% of the apps are designed for social networking, followed by educational apps which amount for 3.3% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the `Category` column of the `android_final` data set (two columns which seem to be related).

In [16]:
display_table(android_final, 1)

FAMILY : 18.9 %
GAME : 9.7 %
TOOLS : 8.5 %
BUSINESS : 4.6 %
LIFESTYLE : 3.9 %
PRODUCTIVITY : 3.9 %
FINANCE : 3.7 %
MEDICAL : 3.5 %
SPORTS : 3.4 %
PERSONALIZATION : 3.3 %
COMMUNICATION : 3.2 %
HEALTH_AND_FITNESS : 3.1 %
PHOTOGRAPHY : 2.9 %
NEWS_AND_MAGAZINES : 2.8 %
SOCIAL : 2.7 %
TRAVEL_AND_LOCAL : 2.3 %
SHOPPING : 2.2 %
BOOKS_AND_REFERENCE : 2.1 %
DATING : 1.9 %
VIDEO_PLAYERS : 1.8 %
MAPS_AND_NAVIGATION : 1.4 %
FOOD_AND_DRINK : 1.2 %
EDUCATION : 1.2 %
ENTERTAINMENT : 1.0 %
LIBRARIES_AND_DEMO : 0.9 %
AUTO_AND_VEHICLES : 0.9 %
HOUSE_AND_HOME : 0.8 %
WEATHER : 0.8 %
EVENTS : 0.7 %
PARENTING : 0.7 %
ART_AND_DESIGN : 0.6 %
COMICS : 0.6 %
BEAUTY : 0.6 %


The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for 19% of the apps) means mostly games for kids.

[![img](https://s3.amazonaws.com/dq-content/350/py1m8_family.png)](https://play.google.com/store/apps/category/FAMILY?hl=en)

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the `Genres` column:

In [17]:
display_table(android_final,-4)

Tools : 8.4 %
Entertainment : 6.1 %
Education : 5.3 %
Business : 4.6 %
Productivity : 3.9 %
Lifestyle : 3.9 %
Finance : 3.7 %
Medical : 3.5 %
Sports : 3.5 %
Personalization : 3.3 %
Communication : 3.2 %
Action : 3.1 %
Health & Fitness : 3.1 %
Photography : 2.9 %
News & Magazines : 2.8 %
Social : 2.7 %
Travel & Local : 2.3 %
Shopping : 2.2 %
Books & Reference : 2.1 %
Simulation : 2.0 %
Dating : 1.9 %
Arcade : 1.9 %
Video Players & Editors : 1.8 %
Casual : 1.8 %
Maps & Navigation : 1.4 %
Food & Drink : 1.2 %
Puzzle : 1.1 %
Racing : 1.0 %
Role Playing : 0.9 %
Libraries & Demo : 0.9 %
Auto & Vehicles : 0.9 %
Strategy : 0.9 %
House & Home : 0.8 %
Weather : 0.8 %
Events : 0.7 %
Adventure : 0.7 %
Comics : 0.6 %
Beauty : 0.6 %
Art & Design : 0.6 %
Parenting : 0.5 %
Card : 0.5 %
Casino : 0.4 %
Trivia : 0.4 %
Educational;Education : 0.4 %
Board : 0.4 %
Educational : 0.4 %
Education;Education : 0.3 %
Word : 0.3 %
Casual;Pretend Play : 0.2 %
Music : 0.2 %
Racing;Action & Adventure : 0.2 %
Puzzle;B

The difference between the `Genres` and the `Category` columns is not crystal clear, but one thing we can notice is that the `Genres` column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the `Category` column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` column.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [18]:
def unique_dict(dataset, index):
    result={}
    for row in dataset:
        name=row[index]
        if row[index] not in result:
            result[name]=0
    return result

def avg_dict(dataset, groupby_index, result_index):
    groupby_dict=unique_dict(dataset, groupby_index)
    for item in groupby_dict:
        total=0
        for row in dataset:
            number=float(row[result_index])
            if item==row[groupby_index]:
                total+=number
        groupby_dict[item]=total
    
    count_dict=freq_table(dataset, groupby_index)
    for key in groupby_dict:
        result=round(groupby_dict[key]/count_dict[key])
        groupby_dict[key]=result
    
    return groupby_dict

In [19]:
ios_pop=avg_dict(ios_final, 11, 5)
for_display=[]

for key in ios_pop:
    as_tuple=(ios_pop[key], key)
    for_display.append(as_tuple)
ios_rank=sorted(for_display, reverse=True)

for item in ios_rank:
    print(item[1], ':', item[0])

Navigation : 86090
Reference : 74942
Social Networking : 71548
Music : 57327
Weather : 52280
Book : 39758
Food & Drink : 33334
Finance : 31468
Photo & Video : 28442
Travel : 28244
Shopping : 26920
Health & Fitness : 23298
Sports : 23009
Games : 22789
News : 21248
Productivity : 21028
Utilities : 18684
Lifestyle : 16486
Entertainment : 14030
Business : 7491
Education : 7004
Catalogs : 4004
Medical : 612


On average, navigation apps have the highest number of user reviews, but we can't decide wether majority apps inside this category enjoy large popularity or it's because a few monopolistic apps with dominant popularity exist.

To check the real situation inside each genre, we'll write the `inspect()` function to help us:

In [20]:
def inspect(genre):
    genre_freq=freq_table(ios_final, 11)
    genre_total=genre_freq[genre]*ios_pop[genre]
    for app in ios_final:
        if app[11]==genre:
            percent=round(float(app[5])/genre_total,2)*100
            print(app[1],':',percent, '%')

As we can see,  avrage `rating_count_tot` is heavily influenced by Waze and Google Maps, which have close to 97% of thr user reviews together.

In [21]:
inspect('Navigation')

Waze - GPS Navigation, Maps & Real-time Traffic : 67.0 %
Google Maps - Navigation & Transit : 30.0 %
Geocaching® : 2.0 %
CoPilot GPS – Car Navigation & Offline Maps : 1.0 %
ImmobilienScout24: Real Estate Search in Germany : 0.0 %
Railway Route Search : 0.0 %


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [22]:
inspect('Reference')

Bible : 73.0 %
Dictionary.com Dictionary & Thesaurus : 15.0 %
Dictionary.com Dictionary & Thesaurus for iPad : 4.0 %
Google Translate : 2.0 %
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 1.0 %
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 1.0 %
Merriam-Webster Dictionary : 1.0 %
Night Sky : 1.0 %
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 1.0 %
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 0.0 %
GUNS MODS for Minecraft PC Edition - Mods Tools : 0.0 %
Guides for Pokémon GO - Pokemon GO News and Cheats : 0.0 %
WWDC : 0.0 %
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 0.0 %
VPN Express : 0.0 %
Real Bike Traffic Rider Virtual Reality Glasses : 0.0 %
教えて!goo : 0.0 %
Jishokun-Japanese English Dictionary & Translator : 0.0 %


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:
- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.
- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.
- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [23]:
display_table(android_final, 5)

1,000,000+ : 15.7 %
100,000+ : 11.6 %
10,000,000+ : 10.5 %
10,000+ : 10.2 %
1,000+ : 8.4 %
100+ : 6.9 %
5,000,000+ : 6.8 %
500,000+ : 5.6 %
50,000+ : 4.8 %
5,000+ : 4.5 %
10+ : 3.5 %
500+ : 3.2 %
50,000,000+ : 2.3 %
100,000,000+ : 2.1 %
50+ : 1.9 %
5+ : 0.8 %
1+ : 0.5 %
500,000,000+ : 0.3 %
1,000,000,000+ : 0.2 %
0+ : 0.0 %
0 : 0.0 %


For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to find out which app genres attract the most users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on. To perform computations, however, we'll need to convert each install number from a string to a float. This means we need to remove the commas and the plus characters, or the conversion will fail and cause an error.

In [24]:
android_pop=unique_dict(android_final, 1)
for key in android_pop:
    total=0
    app_number=0
    for app in android_final:
        genre=app[1]
        installs=app[5]
        installs=installs.replace('+','')
        installs=installs.replace(',','')
        if genre==key:
            total+=float(installs)
            app_number+=1
    avg_installs=round(total/app_number)
    android_pop[key]=avg_installs

for_display2=[]

for key in android_pop:
    as_tuple=(android_pop[key], key)
    for_display2.append(as_tuple)
android_rank=sorted(for_display2, reverse=True)

for item in android_rank:
    print(item[1], ':', item[0])

COMMUNICATION : 38456119
VIDEO_PLAYERS : 24727872
SOCIAL : 23253652
PHOTOGRAPHY : 17840110
PRODUCTIVITY : 16787331
GAME : 15588016
TRAVEL_AND_LOCAL : 13984078
ENTERTAINMENT : 11640706
TOOLS : 10801391
NEWS_AND_MAGAZINES : 9549178
BOOKS_AND_REFERENCE : 8767812
SHOPPING : 7036877
PERSONALIZATION : 5201483
WEATHER : 5074486
HEALTH_AND_FITNESS : 4188822
MAPS_AND_NAVIGATION : 4056942
FAMILY : 3695642
SPORTS : 3638640
ART_AND_DESIGN : 1986335
FOOD_AND_DRINK : 1924898
EDUCATION : 1833495
BUSINESS : 1712290
LIFESTYLE : 1437816
FINANCE : 1387692
HOUSE_AND_HOME : 1331541
DATING : 854029
COMICS : 817657
AUTO_AND_VEHICLES : 647318
LIBRARIES_AND_DEMO : 638504
PARENTING : 542604
BEAUTY : 513152
EVENTS : 253542
MEDICAL : 120551


On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [25]:
for app in android_final:
    if app[1]=='COMMUNICATION' and (app[5]=='1,000,000,000+'
                                    or app[5]=='500,000,000+'
                                    or app[5]=='100,000,000+'):
        print(app[0],':',app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [26]:
under_100_m=[]
for app in android_final:
    name=app[1]
    installs=app[5]
    installs=installs.replace('+','')
    installs=installs.replace(',','')
    if name=='COMMUNICATION' and float(installs)<100000000:
        under_100_m.append(float(installs))

print(sum(under_100_m)/len(under_100_m))

3603485.3884615386


We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [27]:
for app in android_final:
    if app[1]=='BOOKS_AND_REFERENCE':
        print(app[0],':',app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [28]:
for app in android_final:
    if app[1]=='BOOKS_AND_REFERENCE' and (app[5]=='1,000,000,000+' 
                                          or app[5]=='500,000,000+' 
                                          or app[5]=='100,000,000+'):
        print(app[0],':',app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [29]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.